In [ ]:
import pandas as pd
import numpy as np

class userInput:

    def __init__(self, minimumInterEvent,stormDepthCutoff,gaugeName, minimumTotalDepth,maximumTotalDepth):

        self.minimumInterEvent = minimumInterEvent
        self.stormDepthCutoff = stormDepthCutoff

        self.gaugeName = gaugeName

        self.minimumTotalDepth = minimumTotalDepth
        self.maximumTotalDepth = maximumTotalDepth

    def stormCsvReader(self):
        
        self.gaugeDataFrame = pd.read_csv(self.gaugeName)
        self.timeStepIncrement = 1

    def stormMITList(self):

        self.stormList = []

    def huffCurveList(self):

        self.huffCurveRawList = []

    def timeseriesIncrements(self): 

        self.fractionList = np.arange(0, 1 + (1/60), (1/60)).round(5)

    def minimumInterEventStorm(self):

        minimumInterEventCounter = 0
        currentWindowSum = 0
        gaugeRainfallArray = self.gaugeDataFrame['Rainfall (mm)']

        for windowEnd in range(len(gaugeRainfallArray)):

            if minimumInterEventCounter >= self.minimumInterEvent:

                currentWindowStartIndex = windowEnd - currentWindowSum 
                currentWindowEndIndex = windowEnd - self.minimumInterEvent  

                minimumInterEventCounter = 0
                currentWindowSum = 0 
        
                individualStormList = [currentWindowStartIndex,currentWindowEndIndex]
                self.stormList.append(individualStormList)
            
            if gaugeRainfallArray[windowEnd] > self.stormDepthCutoff and minimumInterEventCounter <= self.minimumInterEvent:

                currentWindowSum += 1 
            
            elif gaugeRainfallArray[windowEnd] <= self.stormDepthCutoff:

                if currentWindowSum > 0:

                    if gaugeRainfallArray[windowEnd] < self.stormDepthCutoff:

                        minimumInterEventCounter += 1
                        currentWindowSum += 1

    def huffCurveOperations(self):

        stormIteration = 0

        for eachStorm in self.stormList:

            individualStormDataFrame = self.gaugeDataFrame[eachStorm[0]:eachStorm[1]].copy()
            individualStormDataFrame = individualStormDataFrame.reset_index(drop=True)
            individualStormDataFrame['Date and time'] = pd.to_datetime(individualStormDataFrame['Date and time'], format='%H:%M:%S %d/%m/%Y')

        if individualStormDataFrame.iloc[-1] == 0:

            individualStormDataFrame = individualStormDataFrame.drop(index=individualStormDataFrame.index[-1]) # Drops any values that aren't being picked up in the algorithm.

        timestepStart = pd.DataFrame(
            {'Date and time': [individualStormDataFrame['Date and time'][0] - pd.Timedelta(hours=1)], 
             'Rainfall (mm)':[0],'Quality Code':individualStormDataFrame['Quality Code'][0]})

        cleanedInvidivualStorm = pd.concat([timestepStart, individualStormDataFrame], ignore_index=True)

        totalRainfall = cleanedInvidivualStorm['Rainfall (mm)'].sum()

        if self.minimumTotalDepth <= totalRainfall <= self.maximumTotalDepth:

            cleanedInvidivualStorm['Storm Duration (min)'] = cleanedInvidivualStorm.index * 60
            upperRange = cleanedInvidivualStorm.max() + 1

            cleanedInvidivualStorm.set_index('Storm Duration (min)', inplace=True)
            newIndex = pd.Index(range(0,upperRange,self.timestepIncrement), name='Storm Duration (min)')
            cleanedInvidivualStorm = cleanedInvidivualStorm.reindex(newIndex)

            # Filling missing values.

            cleanedInvidivualStorm['Rainfall (mm)'] = cleanedInvidivualStorm['Rainfall (mm)'].bfill() / 60
            cleanedInvidivualStorm['Quality Code'] = cleanedInvidivualStorm['Quality Code'].bfill()
            cleanedInvidivualStorm['Date and time'] = cleanedInvidivualStorm['Date and time'].interpolate()

            # Calculate Storm Duration

            cleanedInvidivualStorm['Storm Duration (min)'] = cleanedInvidivualStorm.index
            cleanedInvidivualStorm = cleanedInvidivualStorm.reset_index(drop=True)
            cleanedInvidivualStorm['Storm Duration (h)'] = cleanedInvidivualStorm['Storm Duration (min)'] / 60

            # Calculate cumulative rainfall

            cleanedInvidivualStorm['Total Storm Duration (%)'] = cleanedInvidivualStorm['Storm Duration (h)'] / cleanedInvidivualStorm['Storm Duration (h)'].max()
            cleanedInvidivualStorm['Total Storm Duration (%)'] = cleanedInvidivualStorm['Total Storm Duration (%)'].round(5)
            cleanedInvidivualStorm['Cumulative Rainfall (mm)'] = cleanedInvidivualStorm['Rainfall (mm)'].cumsum() 
            cleanedInvidivualStorm['Cumulative Rainfall (%)'] = cleanedInvidivualStorm['Cumulative Rainfall (mm)'] / cleanedInvidivualStorm['Rainfall (mm)'].sum()

            # Filtering by the percentage of storm increment.

            gaugeHuffDataFrame = cleanedInvidivualStorm.loc[cleanedInvidivualStorm['Total Storm Duration (%)'].isin(self.fractionList)]
            stormNameString = ('Storm {iteration}'.format(iteration = stormIteration))
            gaugeHuffDataFrame = gaugeHuffDataFrame.rename(columns={'Cumulative Rainfall (%)': stormNameString})

            self.huffCurveRawList.append(gaugeHuffDataFrame[stormNameString].to_list())

            stormIteration += 1
        
    def huffCurveQuantiles(self):

        self.quantileHuffCurveList = []
    
        for eachQuantile in np.arange(0.1,1,0.1):

            self.quantileHuffCurveList.append(self.huffCurveRawList.quantile(eachQuantile, axis = 0))

        huffQuantiles = pd.DataFrame(self.quantileHuffCurveList).T
        huffQuantiles['Duration (%)'] = self.fractionList










    











